In [32]:
import importlib
import packages.word_corrector.src.core as core

importlib.reload(core)

<module 'packages.word_corrector.src.core' from '/home/jovyan/work/packages/word_corrector/src/core.py'>

In [33]:
from gradio_client import Client, handle_file   
# импорт contenxtmanager
from contextlib import contextmanager
import time
import os
from pydantic import BaseModel, Field
import csv
from rapidfuzz import process, fuzz
import re

from tqdm import tqdm
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import json
from pathlib import Path

In [34]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self  # возвращаем self, чтобы получить доступ к elapsed_time

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.elapsed = round(self.end - self.start,4)
        #print(f"Cell execution time: {self.elapsed:.4f} seconds")


In [35]:
importlib.reload(core)
from packages.word_corrector.src.core import   PhraseCorrectorByWords, PhraseCorrectorNgrams, correct_text

In [36]:
class CSVRow(BaseModel):
    file_name: str
    text: str
    elapsed_time: float 
    corrected_text: str
    elapsed_time_corrected: float
    original_text:str


In [37]:
# Скачиваем тестовый файл для корректора
!wget https://docs.google.com/spreadsheets/d/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY/export?format=xlsx -O data/test_expecting_corrections.xlsx

--2025-05-26 06:45:25--  https://docs.google.com/spreadsheets/d/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 142.251.1.194, 2a00:1450:4010:c1e::c2
Connecting to docs.google.com (docs.google.com)|142.251.1.194|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-14-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/p3g0misr9eb6busg6642v2htqg/1748241925000/117963286932567830463/*/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY?format=xlsx [following]
--2025-05-26 06:45:25--  https://doc-14-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/p3g0misr9eb6busg6642v2htqg/1748241925000/117963286932567830463/*/1IPXCgSj6a2itzLxY64gVVU6UIyAHaAI4X8P5xPdaykY?format=xlsx
Resolving doc-14-1s-sheets.googleusercontent.com (doc-14-1s-sheets.googleusercontent.com)... 64.233.164.132, 2a00:1450:4010:c07::84
Connecting to doc-14-1s-sheets.googleusercontent.com (doc-

In [38]:
df_test_correction = pd.read_excel('data/test_expecting_corrections.xlsx')

In [39]:
df_test_correction

,file_name,text,elapsed_time,corrected_text,elapsed_time_corrected,original_text
0,test1,ТЕСТ_1 ГРАММЫ\n1r\n,NaN,ТЕСТ_1 ГРАММЫ\n1Г\n,NaN,NaN
1,test2,ТЕСТ_2 КИЛОГРАММЫ\n1кг 1кr 1kr 1кr,NaN,ТЕСТ_2 КИЛОГРАММЫ\n1КГ 1КГ 1КГ 1КГ,NaN,NaN
2,test3,ТЕСТ_3 ЛИТРЫ\n1л 1п 1n,NaN,ТЕСТ_3 ЛИТРЫ\n1Л 1Л 1Л,NaN,NaN
3,test4,ТЕСТ_4 МИЛЛИЛИТРЫ\n1мл 1mn 1мn 1mл 1мп,NaN,ТЕСТ_4 МИЛЛИЛИТРЫ\n1МЛ 1МЛ 1МЛ 1МЛ 1МЛ,NaN,NaN
4,test5,ТЕСТ_5 УПАКОВКИ\n1уп 1yп 1yn 1уn 1ул 1yп\n,NaN,ТЕСТ_5 УПАКОВКИ\n1УП 1УП 1УП 1УП 1УП 1УП\n,NaN,NaN
5,test6,ТЕСТ_6 ПУСТЫЕ СТРОКИ\n\nтретья строка,NaN,ТЕСТ_6 ПУСТЫЕ СТРОКИ\nтретья строка,NaN,NaN
6,test7,"ТЕСТ_7 СПЕЦСИМВОЛЫ\n!""№;%:?*().,}{<>",NaN,"ТЕСТ_7 СПЕЦСИМВОЛЫ\n!""№;%:?*().,}{<>",NaN,NaN
7,test8,ТЕСТ_8 НЕ ОБРАБАТЫВАТЬ ТРЕТЬЮ СТРОКУ\nВТОРАЯ С...,NaN,ТЕСТ_8 НЕ ОБРАБАТЫВАТЬ ТРЕТЬЮ СТРОКУ\nВТОРАЯ С...,NaN,NaN
8,names/930.jpg,Молоко КРАСНАЯ ЦЕНА\n800мл,2.055,МОЛОКО КРАСНАЯ ЦЕНА\n800МЛ,0.0017,"Молоко КРАСНАЯ ЦЕНА\n800мл\nмит.паст.2,5%"


# Корректор

In [40]:
corrector = PhraseCorrectorByWords("data/words.txt")

## Проверяем работу корректора


In [41]:
with open(f"results_correction.tsv", "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(CSVRow.model_fields.keys())
    
        for index, row in tqdm(df_test_correction.iterrows(), total=len(df_test_correction)):
            txt = row['text']
            original_text = str(row['original_text'])



            with Timer() as timer_corrected:
                corrected_text , _ = correct_text(txt, corrector)
            elapsed_time_corrected = timer_corrected.elapsed


            csv_row = CSVRow(
                file_name=row['file_name'],
                text=txt,
                elapsed_time=0,
                corrected_text=corrected_text,
                elapsed_time_corrected=elapsed_time_corrected,
                original_text=original_text
    )


            writer.writerow(csv_row.model_dump().values())
            f.flush()

100%|██████████| 9/9 [00:00<00:00, 202.26it/s]


In [42]:
with Timer() as t:
    time.sleep(1)

print(t.elapsed)

1.0001


In [43]:
DATA_BASE_DIR = Path("/home/jovyan/work/data")

In [44]:
with open(DATA_BASE_DIR / "test_dataset.json", "r") as f:
    test_data = json.load(f)

In [45]:
test_data[:1]

[{'image_path': 'names/930.jpg',
  'text': 'Молоко КРАСНАЯ ЦЕНА\n800мл\nмит.паст.2,5%'}]

In [46]:
client = Client("http://qwen_api:7860/")

Loaded as API: http://qwen_api:7860/ ✔


In [47]:
SYSTEM_PROMPT = '''
Ты система распознавания текста, которая распознает текст с русских ценников в супермаркетах. В тексте могут встречаться названия на английском.
Выведи только текст, построчно, в точности как на изображении. Без дополнительной информации. Не переводи слова на другой язык.
'''

In [48]:
def  predict(file_path, model_name = "Qwen2_5-7b"):    
    result = client.predict(
		system_prompt =SYSTEM_PROMPT,
		user_prompt="",
		image=handle_file(file_path),
        #image=None,
		selected_model=model_name,
		temperature=0.2,
		api_name="/predict"
	)
    return result

In [49]:
predict(DATA_BASE_DIR / "names/1.jpg")

'Вода святой источник\n1.0л\nприродная питьевая негаз.ПЭТ'

In [50]:
def mass_experiment(model_name, limit=1, debug=False, prefix=""):
    with open(f"results_{prefix}{model_name}.tsv", "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(CSVRow.model_fields.keys())
    
        for item in tqdm(test_data[:limit], desc="Process images", unit="image"):
            try:
                file_name = item["image_path"]
                original_text = item["text"]
                row = CSVRow(
                    file_name=file_name,
                    text="",
                    elapsed_time=0.0,
                    corrected_text="",
                    elapsed_time_corrected=0.0,
                    original_text=original_text
                )
    
                file_path = DATA_BASE_DIR / file_name
    
                # Чтение изображения
                if debug:
                    try:
                        img = mpimg.imread(file_path)
                        plt.imshow(img)
                        plt.axis('off')
                        plt.show()
                    except Exception as img_error:
                        print(f"[Ошибка изображения] {file_name}: {img_error}")
                        continue  # пропустить эту строку
        
                # Предсказание
                with Timer() as t:
                    result = predict(file_path, model_name)
                row.elapsed_time = t.elapsed
                row.text = result
    
                # Коррекция
                with Timer() as t:
                    corrected_result, _ = correct_text(result, corrector)
                row.elapsed_time_corrected = t.elapsed
                row.corrected_text = corrected_result
    
                # Лог + запись
                if debug:
                    print(json.dumps(row.model_dump(), indent=4, ensure_ascii=False))
                writer.writerow(row.model_dump().values())
                f.flush()
    
            except Exception as e:
                print(f"[Общая ошибка] {item['image_path']}: {e}")
                continue


In [51]:
model_names = [
    "Qwen2_5-7b",
    "Qwen2_5-3b_finetuned",
    "Qwen2_5-7b_finetuned",
    "Qwen2-vl-7b_finetuned",
    "Qwen2_5-3b",
    "Qwen2-vl-7b"
]

# Запуск экспериментов

In [ ]:
for model_name in model_names:
    print(model_name)
    mass_experiment(model_name, -1, False, "")

Qwen2_5-7b


Process images:   2%|▏         | 3/198 [00:04<03:42,  1.14s/image]

[Общая ошибка] names/577.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images:  25%|██▌       | 50/198 [01:55<05:15,  2.13s/image]

[Общая ошибка] names/524.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images:  39%|███▉      | 77/198 [03:03<05:42,  2.83s/image]